# Dynamic Breakpoints
## Review
We discussed motivations for human-in-the-loop:

1. **Approval** - We can interrupt our agent, surface state to a user, and allow the user to accept an action

2. **Debugging** - We can rewind the graph to reproduce or avoid issues

3. **Editing** - You can modify the state

We covered breakpoints as a general way to stop the graph at specific steps, which enables use-cases like Approval

We also showed how to edit graph state, and introduce human feedback.

## Goals for this session
**Breakpoints** are set by the developer on a specific node during graph compilation.

***But, sometimes it is helpful to allow the graph dynamically interrupt itself!***

This is an internal breakpoint, and can be achieved using **NodeInterrupt**.

This has a few specific benefits:

1. You can do it **conditionally** (from inside a node based on developer-defined logic).

2. You can **communicate** to the user why its interrupted (by passing whatever you want to the **NodeInterrupt**).

Let's create a graph where a **NodeInterrupt** is thrown based upon length of the input.

In [ ]:
from IPython.display import Image, display

from typing_extensions import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.errors import NodeInterrupt
from langgraph.graph import START, END, StateGraph

class State(TypedDict):
    input: str

def step_1(state: State) -> State:
    print("---Step 1---")
    return state

def step_2(state: State) -> State:
    # Let's optionally raise a NodeInterrupt if the length of the input is longer than 5 characters
    if len(state['input']) > 5:
        raise NodeInterrupt(f"Received input that is longer than 5 characters: {state['input']}")
    
    print("---Step 2---")
    return state

def step_3(state: State) -> State:
    print("---Step 3---")
    return state

builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

# Set up memory
memory = MemorySaver()

# Compile the graph with memory
graph = builder.compile(checkpointer=memory)

# View
display(Image(graph.get_graph().draw_mermaid_png()))

Let's run the graph with an input that's longer than 5 characters.

In [4]:
initial_input = {"input": "hello world"}
thread_config = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread_config, stream_mode="values"):
    print(event)

{'input': 'hello world'}
---Step 1---
{'input': 'hello world'}


If we inspect the graph state at this point, we the node set to execute next (```step_2```).

In [ ]:
state = graph.get_state(thread_config)
print(state.next)

We can see that the ```Interrupt``` is logged to state.

In [ ]:
print(state.tasks)

We can try to resume the graph from the breakpoint.

But, this just re-runs the same node!

Unless state is changed we will be stuck here.

In [6]:
for event in graph.stream(None, thread_config, stream_mode="values"):
    print(event)

{'input': 'hello world'}


In [7]:
state = graph.get_state(thread_config)
print(state.next)

('step_2',)


Now, we can update state

In [ ]:
graph.update_state(
    thread_config,
    {"input": "hi"},
)

In [9]:
for event in graph.stream(None, thread_config, stream_mode="values"):
    print(event)

{'input': 'hi'}
---Step 2---
{'input': 'hi'}
---Step 3---
{'input': 'hi'}
